In [1]:
"""A UI solution and host service to interact with the agent framework.
run:
  uv main.py
"""
import os,sys
sys.path.insert(0, r"E:\Github\a2a-samples\samples\python")


from contextlib import asynccontextmanager

import httpx
import mesop as me

from components.api_key_dialog import api_key_dialog
from components.page_scaffold import page_scaffold
from dotenv import load_dotenv
from fastapi import FastAPI
from fastapi.middleware.wsgi import WSGIMiddleware
from pages.agent_list import agent_list_page
from pages.conversation import conversation_page
from pages.event_list import event_list_page
from pages.home import home_page_content
from pages.settings import settings_page_content
from pages.task_list import task_list_page
from service.server.server import ConversationServer
from state import host_agent_service
from state.state import AppState


load_dotenv()


def on_load(e: me.LoadEvent):  # pylint: disable=unused-argument
    """On load event"""
    state = me.state(AppState)
    me.set_theme_mode(state.theme_mode)
    if 'conversation_id' in me.query_params:
        state.current_conversation_id = me.query_params['conversation_id']
    else:
        state.current_conversation_id = ''

    # check if the API key is set in the environment
    # and if the user is using Vertex AI
    uses_vertex_ai = (
        os.getenv('GOOGLE_GENAI_USE_VERTEXAI', '').upper() == 'TRUE'
    )
    api_key = os.getenv('GOOGLE_API_KEY', '')

    if uses_vertex_ai:
        state.uses_vertex_ai = True
    elif api_key:
        state.api_key = api_key
    else:
        # Show the API key dialog if both are not set
        state.api_key_dialog_open = True


# Policy to allow the lit custom element to load
security_policy = me.SecurityPolicy(
    allowed_script_srcs=[
        'https://cdn.jsdelivr.net',
    ]
)


@me.page(
    path='/',
    title='Chat',
    on_load=on_load,
    security_policy=security_policy,
)
def home_page():
    """Main Page"""
    state = me.state(AppState)
    # Show API key dialog if needed
    api_key_dialog()
    with page_scaffold():  # pylint: disable=not-context-manager
        home_page_content(state)


@me.page(
    path='/agents',
    title='Agents',
    on_load=on_load,
    security_policy=security_policy,
)
def another_page():
    """Another Page"""
    api_key_dialog()
    agent_list_page(me.state(AppState))


@me.page(
    path='/conversation',
    title='Conversation',
    on_load=on_load,
    security_policy=security_policy,
)
def chat_page():
    """Conversation Page."""
    api_key_dialog()
    conversation_page(me.state(AppState))


@me.page(
    path='/event_list',
    title='Event List',
    on_load=on_load,
    security_policy=security_policy,
)
def event_page():
    """Event List Page."""
    api_key_dialog()
    event_list_page(me.state(AppState))


@me.page(
    path='/settings',
    title='Settings',
    on_load=on_load,
    security_policy=security_policy,
)
def settings_page():
    """Settings Page."""
    api_key_dialog()
    settings_page_content()


@me.page(
    path='/task_list',
    title='Task List',
    on_load=on_load,
    security_policy=security_policy,
)
def task_page():
    """Task List Page."""
    api_key_dialog()
    task_list_page(me.state(AppState))


class HTTPXClientWrapper:
    """Wrapper to return the singleton client where needed."""

    async_client: httpx.AsyncClient = None

    def start(self):
        """Instantiate the client. Call from the FastAPI startup hook."""
        self.async_client = httpx.AsyncClient(timeout=30)

    async def stop(self):
        """Gracefully shutdown. Call from FastAPI shutdown hook."""
        await self.async_client.aclose()
        self.async_client = None

    def __call__(self):
        """Calling the instantiated HTTPXClientWrapper returns the wrapped singleton."""
        # Ensure we don't use it if not started / running
        assert self.async_client is not None
        return self.async_client


# Setup the server global objects
httpx_client_wrapper = HTTPXClientWrapper()
agent_server = None


@asynccontextmanager
async def lifespan(app: FastAPI):
    httpx_client_wrapper.start()
    agent_server = ConversationServer(app, httpx_client_wrapper())
    app.openapi_schema = None
    app.mount(
        '/',
        WSGIMiddleware(
            me.create_wsgi_app(
                debug_mode=os.environ.get('DEBUG_MODE', '') == 'true'
            )
        ),
    )
    app.setup()
    yield
    await httpx_client_wrapper.stop()


if __name__ == '__main__':
    import uvicorn

    app = FastAPI(lifespan=lifespan)

    # Setup the connection details, these should be set in the environment
    host = os.environ.get('A2A_UI_HOST', '0.0.0.0')
    port = int(os.environ.get('A2A_UI_PORT', '12000'))

    # Set the client to talk to the server
    host_agent_service.server_url = f'http://{host}:{port}'

    uvicorn.run(
        app,
        host=host,
        port=port,
        timeout_graceful_shutdown=0,
    )


RuntimeError: asyncio.run() cannot be called from a running event loop

In [2]:
import os, sys
import nest_asyncio
import asyncio

from fastapi import FastAPI
from fastapi.middleware.wsgi import WSGIMiddleware
import httpx

# 🧩 本地导入路径，根据你放置的项目结构调整
sys.path.insert(0, r"E:\Github\a2a-samples\samples\python")

from components.api_key_dialog import api_key_dialog
from components.page_scaffold import page_scaffold
from pages.agent_list import agent_list_page
from pages.conversation import conversation_page
from pages.event_list import event_list_page
from pages.home import home_page_content
from pages.settings import settings_page_content
from pages.task_list import task_list_page
from service.server.server import ConversationServer
from state import host_agent_service
from state.state import AppState
import mesop as me
from contextlib import asynccontextmanager
from dotenv import load_dotenv

# ✅ 允许在 Notebook 中多次运行 asyncio
nest_asyncio.apply()

# ✅ 加载 .env 环境变量
load_dotenv()

# ⚙️ 初始化 FastAPI & 客户端封装
class HTTPXClientWrapper:
    async_client: httpx.AsyncClient = None

    def start(self):
        self.async_client = httpx.AsyncClient(timeout=30)

    async def stop(self):
        await self.async_client.aclose()
        self.async_client = None

    def __call__(self):
        assert self.async_client is not None
        return self.async_client


httpx_client_wrapper = HTTPXClientWrapper()

# ✅ 页面注册函数（照搬原代码结构）
def on_load(e: me.LoadEvent):
    state = me.state(AppState)
    me.set_theme_mode(state.theme_mode)
    if 'conversation_id' in me.query_params:
        state.current_conversation_id = me.query_params['conversation_id']
    else:
        state.current_conversation_id = ''
    uses_vertex_ai = os.getenv('GOOGLE_GENAI_USE_VERTEXAI', '').upper() == 'TRUE'
    api_key = os.getenv('GOOGLE_API_KEY', '')
    if uses_vertex_ai:
        state.uses_vertex_ai = True
    elif api_key:
        state.api_key = api_key
    else:
        state.api_key_dialog_open = True

security_policy = me.SecurityPolicy(allowed_script_srcs=['https://cdn.jsdelivr.net'])

@me.page(path='/', title='Chat', on_load=on_load, security_policy=security_policy)
def home_page():
    api_key_dialog()
    with page_scaffold():
        home_page_content(me.state(AppState))

@me.page(path='/agents', title='Agents', on_load=on_load, security_policy=security_policy)
def agents_page():
    api_key_dialog()
    agent_list_page(me.state(AppState))

@me.page(path='/conversation', title='Conversation', on_load=on_load, security_policy=security_policy)
def conversation():
    api_key_dialog()
    conversation_page(me.state(AppState))

@me.page(path='/event_list', title='Event List', on_load=on_load, security_policy=security_policy)
def events():
    api_key_dialog()
    event_list_page(me.state(AppState))

@me.page(path='/task_list', title='Task List', on_load=on_load, security_policy=security_policy)
def tasks():
    api_key_dialog()
    task_list_page(me.state(AppState))

@me.page(path='/settings', title='Settings', on_load=on_load, security_policy=security_policy)
def settings():
    api_key_dialog()
    settings_page_content()

# ✅ 定义 lifespan 生命周期（FastAPI 启动钩子）
@asynccontextmanager
async def lifespan(app: FastAPI):
    httpx_client_wrapper.start()
    ConversationServer(app, httpx_client_wrapper())
    app.openapi_schema = None
    app.mount("/", WSGIMiddleware(me.create_wsgi_app(debug_mode=False)))
    app.setup()
    yield
    await httpx_client_wrapper.stop()


In [3]:
import uvicorn
from fastapi import FastAPI

app = FastAPI(lifespan=lifespan)

# 环境变量设置（可手动设置）
host = os.environ.get('A2A_UI_HOST', '127.0.0.1')
port = int(os.environ.get('A2A_UI_PORT', '12000'))

host_agent_service.server_url = f"http://{host}:{port}"

# 启动服务
config = uvicorn.Config(app=app, host=host, port=port, timeout_graceful_shutdown=0)
server = uvicorn.Server(config)

await server.serve()


INFO:     Started server process [15532]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)


INFO:     127.0.0.1:5116 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:5116 - "GET /styles.css HTTP/1.1" 304 Not Modified
INFO:     127.0.0.1:5117 - "GET /zone.js/bundles/zone.umd.js HTTP/1.1" 304 Not Modified
INFO:     127.0.0.1:5118 - "GET /prod_bundle.js HTTP/1.1" 304 Not Modified
INFO:     127.0.0.1:5118 - "POST /__ui__ HTTP/1.1" 200 OK
INFO:     127.0.0.1:5118 - "GET /__web-components-module__/components/async_poller.js HTTP/1.1" 304 Not Modified
INFO:     127.0.0.1:5118 - "POST /__ui__ HTTP/1.1" 200 OK
INFO:     127.0.0.1:5127 - "POST /conversation/list HTTP/1.1" 200 OK
INFO:     127.0.0.1:5133 - "POST /task/list HTTP/1.1" 200 OK
INFO:     127.0.0.1:5136 - "POST /message/pending HTTP/1.1" 200 OK
INFO:     127.0.0.1:5118 - "POST /__ui__ HTTP/1.1" 200 OK
INFO:     127.0.0.1:5140 - "POST /conversation/list HTTP/1.1" 200 OK
INFO:     127.0.0.1:5142 - "POST /task/list HTTP/1.1" 200 OK
INFO:     127.0.0.1:5144 - "POST /message/pending HTTP/1.1" 200 OK
INFO:     127.0.0.1:5118 - "POST /

INFO:     Shutting down
ERROR:    Cancel 1 running task(s), timeout graceful shutdown exceeded
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [15532]
ERROR:    ASGI callable returned without completing response.


http error Server error '502 Bad Gateway' for url 'http://127.0.0.1:12000/message/pending'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/502
Error getting pending messages HTTP Error 502: Server error '502 Bad Gateway' for url 'http://127.0.0.1:12000/message/pending'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/502


In [1]:
# ✅ 导入必要模块
import os, sys, asyncio, nest_asyncio
from fastapi import FastAPI
from fastapi.middleware.wsgi import WSGIMiddleware
import httpx
from contextlib import asynccontextmanager
from dotenv import load_dotenv

import mesop as me

# ✅ 本地路径（根据你本机项目结构调整）
sys.path.insert(0, r"E:\Github\a2a-samples\samples\python")

# ✅ 导入组件与状态管理
from components.api_key_dialog import api_key_dialog
from components.page_scaffold import page_scaffold
from pages.agent_list import agent_list_page
from pages.conversation import conversation_page
from pages.event_list import event_list_page
from pages.home import home_page_content
from pages.settings import settings_page_content
from pages.task_list import task_list_page
from service.server.server import ConversationServer
from state import host_agent_service
from state.state import AppState
from state.host_agent_service import UpdateAppState, ListConversations

# ✅ 启用 asyncio 嵌套执行，加载 .env
nest_asyncio.apply()
load_dotenv()

# ✅ HTTP 客户端封装类
class HTTPXClientWrapper:
    async_client: httpx.AsyncClient = None

    def start(self):
        self.async_client = httpx.AsyncClient(timeout=30)

    async def stop(self):
        await self.async_client.aclose()
        self.async_client = None

    def __call__(self):
        assert self.async_client is not None
        return self.async_client

httpx_client_wrapper = HTTPXClientWrapper()

# ✅ 页面加载时自动初始化状态
async def on_load(e: me.LoadEvent):
    state = me.state(AppState)
    me.set_theme_mode(state.theme_mode)

    if 'conversation_id' in me.query_params:
        state.current_conversation_id = me.query_params['conversation_id']
    else:
        state.current_conversation_id = ''

    uses_vertex_ai = os.getenv('GOOGLE_GENAI_USE_VERTEXAI', '').upper() == 'TRUE'
    api_key = os.getenv('GOOGLE_API_KEY', '')
    if uses_vertex_ai:
        state.uses_vertex_ai = True
    elif api_key:
        state.api_key = api_key
    else:
        state.api_key_dialog_open = True

    try:
        await UpdateAppState(state, state.current_conversation_id)
        state.conversations = await ListConversations()
        print("✅ App 状态加载完成")
    except Exception as ex:
        print("⚠️ 状态加载失败:", ex)

# ✅ 安全策略
security_policy = me.SecurityPolicy(allowed_script_srcs=['https://cdn.jsdelivr.net'])

# ✅ 注册页面（注意必须在 app 创建之前注册！）
@me.page(path='/', title='Chat', on_load=on_load, security_policy=security_policy)
def home():
    api_key_dialog()
    with page_scaffold():
        home_page_content(me.state(AppState))

@me.page(path='/agents', title='Agents', on_load=on_load, security_policy=security_policy)
def agents():
    api_key_dialog()
    agent_list_page(me.state(AppState))

@me.page(path='/conversation', title='Conversation', on_load=on_load, security_policy=security_policy)
def conv_page():
    api_key_dialog()
    conversation_page(me.state(AppState))

@me.page(path='/event_list', title='Event List', on_load=on_load, security_policy=security_policy)
def events():
    api_key_dialog()
    event_list_page(me.state(AppState))

@me.page(path='/task_list', title='Task List', on_load=on_load, security_policy=security_policy)
def tasks():
    api_key_dialog()
    task_list_page(me.state(AppState))

@me.page(path='/settings', title='Settings', on_load=on_load, security_policy=security_policy)
def settings():
    api_key_dialog()
    settings_page_content()

# ✅ 生命周期：HTTPX 客户端与 Mesop 绑定
@asynccontextmanager
async def lifespan(app: FastAPI):
    httpx_client_wrapper.start()
    ConversationServer(app, httpx_client_wrapper())
    app.openapi_schema = None
    app.mount("/", WSGIMiddleware(me.create_wsgi_app(debug_mode=True)))
    app.setup()
    yield
    await httpx_client_wrapper.stop()

# ✅ 创建 FastAPI 实例（必须放在页面注册之后）
app = FastAPI(lifespan=lifespan)

# ✅ 启动服务
import uvicorn

host = os.environ.get('A2A_UI_HOST', '127.0.0.1')
port = int(os.environ.get('A2A_UI_PORT', '12000'))
host_agent_service.server_url = f"http://{host}:{port}"

config = uvicorn.Config(app=app, host=host, port=port, timeout_graceful_shutdown=0)
server = uvicorn.Server(config)

await server.serve()


INFO:     Started server process [17500]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)


INFO:     127.0.0.1:8519 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:8520 - "GET /styles.css HTTP/1.1" 200 OK
INFO:     127.0.0.1:8521 - "GET /zone.js/bundles/zone.umd.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:8519 - "GET /editor_bundle/bundle.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:8521 - "POST /__ui__ HTTP/1.1" 200 OK


c:\Users\Laptop\.conda\envs\pytorch\Lib\site-packages\flask\helpers.py:125: RuntimeWarning: coroutine 'on_load' was never awaited
  yield from gen
INFO:     Shutting down
ERROR:    Cancel 1 running task(s), timeout graceful shutdown exceeded
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [17500]
ERROR:    ASGI callable returned without starting response.


INFO:     127.0.0.1:8519 - "GET /__hot-reload__?counter=0 HTTP/1.1" 500 Internal Server Error
